In [1]:
!pip install modelzoo-client[transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.5 MB/s 
     |████████████████████████████████| 82 kB 1.2 MB/s 
     |████████████████████████████████| 789 kB 32.1 MB/s 
     |████████████████████████████████| 4.7 MB 53.3 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.6 MB/s 
     |████████████████████████████████| 596 kB 57.0 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803699 sha256=9d2a78bb569720b4d5875bbb4a1c55520ea6ee41a05ec9725e5c0fb13804ffaf
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: click
    Found existing installation: click 

In [2]:
from google.colab import drive
drive.mount('drive')
!unzip "/content/drive/MyDrive/finetune_gpt_barneyrobin_medium.zip"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Archive:  /content/drive/MyDrive/finetune_gpt_barneyrobin_medium.zip
   creating: content/output-medium/
  inflating: content/output-medium/training_args.bin  
  inflating: content/output-medium/vocab.json  
  inflating: content/output-medium/merges.txt  
  inflating: content/output-medium/config.json  
 extracting: content/output-medium/eval_results.txt  
  inflating: content/output-medium/pytorch_model.bin  
  inflating: content/output-medium/special_tokens_map.json  
   creating: content/output-medium/checkpoint-3500/
  inflating: content/output-medium/checkpoint-3500/scheduler.pt  
  inflating: content/output-medium/checkpoint-3500/training_args.bin  
  inflating: content/output-medium/checkpoint-3500/vocab.json  
  inflating: content/output-medium/checkpoint-3500/merges.txt  
  inflating: content/output-medium/checkpoint-3500/optimizer.pt  
  inflating: content/output-med

In [3]:
!unzip "/content/drive/MyDrive/finetune_gpt_tedlily_medium.zip"

Archive:  /content/drive/MyDrive/finetune_gpt_tedlily_medium.zip
   creating: content/output-medium/
  inflating: content/output-medium/merges.txt  
  inflating: content/output-medium/config.json  
 extracting: content/output-medium/eval_results.txt  
   creating: content/output-medium/checkpoint-3500/
  inflating: content/output-medium/checkpoint-3500/merges.txt  
  inflating: content/output-medium/checkpoint-3500/config.json  
  inflating: content/output-medium/checkpoint-3500/scheduler.pt  
  inflating: content/output-medium/checkpoint-3500/special_tokens_map.json  
  inflating: content/output-medium/checkpoint-3500/vocab.json  
  inflating: content/output-medium/checkpoint-3500/optimizer.pt  
  inflating: content/output-medium/checkpoint-3500/tokenizer_config.json  
  inflating: content/output-medium/checkpoint-3500/training_args.bin  
  inflating: content/output-medium/checkpoint-3500/pytorch_model.bin  
  inflating: content/output-medium/checkpoint-3500/tokenizer.json  
  inflati

In [4]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange
from pathlib import Path

In [6]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
model_barneyrobin = AutoModelWithLMHead.from_pretrained('content/output-medium-barneyrobin')
model_tedlily = AutoModelWithLMHead.from_pretrained('content/output-medium-tedlily')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Let's check one of the models by talking to it ourselves

In [ ]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)
# append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
# generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model_robin.generate(
        bot_input_ids,
        max_length=1000,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # pretty print last ouput tokens from bot
    print("Robin: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hi Robin. how is your new job at World Wide news?
Robin: Oh, I like it. It's great.
>> User:Do you like Sandy Rivers?
Robin: Yeah, he's a pretty cool guy.
>> User:are you sure?
Robin: Yeah, he's really cool. I mean, he's really cool-- but I'm not gonna lie, I'm kind of jealous.
>> User:why are you jealous?
Robin: Uh, because I'm not a billionaire.
>> User:thank you for your honesty


KeyboardInterrupt: ignored

We can see above that the model has reached to its limit - after 4 concat iterations the model got stuck. we will solve it later.  
Next step is to generate a script between Robin and Barney:

In [ ]:
seed = "Maclaren's pub tonight??"
seed_input_ids = tokenizer.encode(seed + tokenizer.eos_token, return_tensors='pt')
barney_input_ids = seed_input_ids

barney_chat_history_ids = model_barney.generate(
        barney_input_ids,
        max_length=1000,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id
    )
barneys_answer = tokenizer.decode(barney_chat_history_ids[:, barney_input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"Seed: {seed}")
print(f"Barney: {barneys_answer}")


for step in range(8):
      new_barney_input_ids = tokenizer.encode(barneys_answer + tokenizer.eos_token, return_tensors='pt')
      robin_input_ids = torch.cat([robin_chat_history_ids, new_barney_input_ids], dim=-1) if step > 0 else new_barney_input_ids
      robin_chat_history_ids = model_robin.generate(
        robin_input_ids,
        max_length=1000,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id
    )
      robins_answer = tokenizer.decode(robin_chat_history_ids[:, robin_input_ids.shape[-1]:][0], skip_special_tokens=True)
      print(f"Robin: {robins_answer}")



      new_robin_input_ids = tokenizer.encode(robins_answer + tokenizer.eos_token, return_tensors='pt')
      barney_input_ids = torch.cat([barney_chat_history_ids, new_robin_input_ids], dim=-1) 
      barney_chat_history_ids = model_barney.generate(
        barney_input_ids,
        max_length=1000,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id
    )
      barneys_answer = tokenizer.decode(barney_chat_history_ids[:, barney_input_ids.shape[-1]:][0], skip_special_tokens=True)
      print(f"Barney: {barneys_answer}")

Seed: Maclaren's pub tonight??
Barney: No, no, no, no, no, no, no, no, no, no, no!
Robin: You're right. Ted, the guy with the stupid hat is wrong. That's it. You're wrong. I am not a lawyer. You're a lawyer.
Barney: Well, you're an idiot.
Robin: Lily is in the bathroom.
Barney: Lily?
Robin: Yes, Lily.


KeyboardInterrupt: ignored

Great! we have generated a script of dialogue between two transformers trained on two characters.   
We liked it that we gave them a seed which was a reference to their favorite bar and they've had the drunkest conversation!

Finally, we will fix the problem mentioned above - its too hard for the model to concatenate past responses more than 3 times. We fixed that by implementing a queue which will make it so that at the most, only the last 3 responses by the adversary model will be considered for each text generation. This will allow us to generate a longer dialogue script.

In [7]:
def enqueue_keep_n(queue, n, item):
  size = len(queue)
  if size < n:
    queue.append(item)
  else:
    queue.pop(0)
    queue.append(item)
  return queue


def tokenize_queue(queue, my_tokenizer):
  if len(queue) == 1:
    return tokenizer.encode(queue[0] + tokenizer.eos_token, return_tensors='pt')
  elif len(queue) == 2:
    first = tokenizer.encode(queue[0] + tokenizer.eos_token, return_tensors='pt')
    second = tokenizer.encode(queue[1] + tokenizer.eos_token, return_tensors='pt')
    return torch.cat([first,second],dim=-1)
  else:
    first = tokenizer.encode(queue[0] + tokenizer.eos_token, return_tensors='pt')
    second = tokenizer.encode(queue[1] + tokenizer.eos_token, return_tensors='pt')
    third = tokenizer.encode(queue[2] + tokenizer.eos_token, return_tensors='pt')
    return torch.cat([first,second,third],dim=-1)

In [15]:
seed = "How are you feeling today?"
seed_input_ids = tokenizer.encode(seed + tokenizer.eos_token, return_tensors='pt')
barneyrobin_input_ids = seed_input_ids

barneyrobin_chat_history_ids = model_barneyrobin.generate(
        barneyrobin_input_ids,
        max_length=1000,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id
    )
barneyrobins_answer = tokenizer.decode(barneyrobin_chat_history_ids[:, barneyrobin_input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"Seed: {seed}")
print(f"BarneyRobin: {barneyrobins_answer}")

# initiate queues
barneyrobin_history_queue = []
tedlily_history_queue = []

for step in range(8):
  enqueue_keep_n(barneyrobin_history_queue, 3, barneyrobins_answer)
  tedlily_input_ids = tokenize_queue(barneyrobin_history_queue, tokenizer) 
  tedlily_chat_history_ids = model_tedlily.generate(
    tedlily_input_ids,
    max_length=1000,
    do_sample=True,
    top_p=0.95,
    top_k=0,
    temperature=0.75,
    pad_token_id=tokenizer.eos_token_id
)
  tedlilys_answer = tokenizer.decode(tedlily_chat_history_ids[:, tedlily_input_ids.shape[-1]:][0], skip_special_tokens=True)
  print(f"TedLily: {tedlilys_answer}")

  enqueue_keep_n(tedlily_history_queue, 3, tedlilys_answer)
  barneyrobin_input_ids = tokenize_queue(tedlily_history_queue, tokenizer) 

  barneyrobin_chat_history_ids = model_barneyrobin.generate(
    barneyrobin_input_ids,
    max_length=1000,
    do_sample=True,
    top_p=0.95,
    top_k=0,
    temperature=0.75,
    pad_token_id=tokenizer.eos_token_id
)
  barneyrobins_answer = tokenizer.decode(barneyrobin_chat_history_ids[:, barneyrobin_input_ids.shape[-1]:][0], skip_special_tokens=True)
  print(f"BarneyRobin: {barneyrobins_answer}")

Seed: How are you feeling today?
BarneyRobin: I feel like I'm wearing nothing at all, Ted. I'm nothing but a shell of what I once was. I know that you're jealous.
TedLily: Well, you know, as long as you're happy, I'm happy.
BarneyRobin: You're going to get married?
TedLily: What? No!
BarneyRobin: But you can't tell Lily your feelings. She's going to get jealous. That's the only way she'll ever be jealous of you.
TedLily: That's not true.
BarneyRobin: Yes, it is. Look, I'm happy for you, Robin. But it doesn't mean I'm happy for the other person.
TedLily: But if they are happy for each other, why can't you be?
BarneyRobin: Because I'm not happy for them. I'm happy for Robin.
TedLily: They're not happy for each other. They're happy for Robin.
BarneyRobin: You're right. They're not happy for each other.
TedLily: Maybe that's the problem.
BarneyRobin: Maybe.
TedLily: Robin and I are just friends. She's not really mad at me.
BarneyRobin: Oh, she mad at you, Ted. She's mad at... you.
TedLily: